# Testing using waveforms

In this notebook, you will see how to use the PeekPokeTester and the most powerful tool for debugging hardware; the waveform. When runnning the PeekPokeTester, we can either inspect the outputs which we peek, but this does not tell us a lot about what is actually happenning. Therefore, it is also possible to output a VCD (Value Change Dump) file, which contains the waveform for the generated hardware signals. This allows us to find exactly which signal is behaving wrong and we can then backtrack to find the source of the problem.

## Setup Chisel

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

## Generating a VCD file
To show how to generate a VCD file, we will use a simple counter with enable as an example:

In [ ]:
class Counter() extends Module {
    val io = IO(new Bundle {
        val out = Output(UInt(4.W)) //The counter has 4 bits of output.
    })
    
    
    val cntEnable = RegInit(10.U)
    val cntCounter= RegInit(0.U(4.W))
    
    cntEnable := Mux(cntEnable === 10.U, 0.U, cntEnable + 1.U)
    
    when(cntEnable === 10.U){
        cntCounter:= cntCounter + 1.U
    }
    
    io.out := 0.U
}

We can now use the PeekPokeTester to run this program. As this is a simple counter without inputs, we will not have anything to Poke. When calling the PeekPokeTester, we include three arguments:

* "--generate-vcd-output","on"
    * Tells the backend to generate a VCD file.
* "-tn", "Counter"
    * TopName, tells the name of the top module.
    * This is only used because we run in Jupyter Notebook, which would otherwise give a mangled name.
* "-target-dir", "generated"
    * Choose the folder/directory to save the output in.

In [ ]:
val testResult = Driver.execute(Array("--generate-vcd-output","on", "-tn", "Counter","--target-dir", "generated"),() => new Counter()) {
  c => new PeekPokeTester(c) {
    step(50) //Move 50 clock cycle ahead.
  }
}

# Verifying with GTKWave
We can now look into the *generated* folder, which contains the Counter.vcd file. This has the waveforms and can be opened in a program like GTKWave. Open the file using GTKWave and verify the result.

## Check the Waveform

To show a waveform, choose the module on the left side, mark the signals you want to look at and click insert.

![GTKWave image](images/GTKWave.png)

You now have a set of signals you can look into.